## Setup

In [7]:
import os
from PIL import Image
import shutil
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np

### Dataset Setup

In [2]:
# Dataset structure setup
source_directory = './data/'
destination_directory = './data/nnUNet_data/Dataset001_ISIC2018'

counter = 0
for filename in os.listdir(source_directory + 'ISIC2018_Task1-2_Training_Input'):
    if filename.split('.')[1] == 'jpg':
        train_file_name = 'isic_' + str(counter).zfill(4) + '_0000.png'
        train_destination_file = os.path.join(destination_directory, 'imagesTr', train_file_name)
        with Image.open(os.path.join(source_directory + 'ISIC2018_Task1-2_Training_Input', filename)) as img:
            img.save(train_destination_file, 'PNG')
        test_file_name = filename.split('.')[0] + '_segmentation.png'
        test_destination_file = os.path.join(destination_directory, 'labelsTr', 'isic_' + str(counter).zfill(4) + '.png')
        shutil.copy(source_directory + 'ISIC2018_Task1_Training_GroundTruth/' + test_file_name, test_destination_file)
        counter += 1

In [8]:
# Labels setup
source_directory = './data/nnUNet_data/Dataset001_ISIC2018/labelsTr_255'
destination_directory = './data/nnUNet_data/Dataset001_ISIC2018/labelsTr'
for filename in os.listdir(source_directory):
    label_image = Image.open(os.path.join(source_directory, filename))
    label_array = np.array(label_image)
    transformed_label_array = label_array // 255
    transformed_label_image = Image.fromarray(transformed_label_array.astype(np.uint8))
    transformed_label_image.save(os.path.join(destination_directory, filename))

## nnUNet V2

In [ ]:
# After setting up the dataset, run below to train the model
export nnUNet_raw="/home/leo/MI_project/data/nnUNet_data"
export nnUNet_preprocessed="/home/leo/MI_project/data/nnUNet_preprocessed"
export nnUNet_results="/home/leo/MI_project/data/nnUNet_results"
export nnUNet_n_proc_DA=18

nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity

nnUNetv2_train 001 2d 0 --npz
nnUNetv2_train 001 2d 1 --npz
nnUNetv2_train 001 2d 2 --npz
nnUNetv2_train 001 2d 3 --npz
nnUNetv2_train 001 2d 4 --npz

## MedSegDiff

In [ ]:
# After setting up the dataset, run below to train the model
python scripts/segmentation_train.py --data_name ISIC --data_dir ./data/MedSegDiff/ISIC2018_Task1-2_Training_Input --out_dir ./data/MedSegDiff/train_results --image_size 256 --num_channels 128 --class_cond False --num_res_blocks 2 --num_heads 1 --learn_sigma True --use_scale_shift_norm False --attention_resolutions 16 --diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False --lr 1e-4 --batch_size 16
python scripts/segmentation_env.py --inp_pth ./data/MedSegDiff/ISIC2018_Task1-2_Test_Input --out_pth test_results
